# Analyse textuelle des publications Facebook des médias québécois en 2022-23

Cette analyse porte sur les publications Facebook plus récentes des médias du Québec (entre le 1<sup>er</sup> janvier 2022 et le 31 mai 2023).

On commence par importer les modules qui nous seront nécessaires et on charge dans la variable `tal` le petit modèle de langue française de spacy (`fr_core_news_sm`).

In [15]:
import pandas as pan
import spacy, csv, numpy
from collections import Counter
tal = spacy.load("fr_core_news_sm")

In [22]:
# fb = pan.read_csv("FB_22_23.csv", low_memory=False, index_col=[0])
fb = pan.read_csv("FB_22_23.csv", low_memory=False)

On constate que les quelque 300 pages Facebook de médias québécois ont publié plus de 780&nbsp;000 *posts* entre le 1er janvier 2022 et le 31 mai 2023.

In [23]:
fb

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),année,mois,inter,jour
0,MSN Québec,msncanadafr,196774760345640,WEBSITE,US,"MSN.ca, le portail québécois où vous informer ...",2011-03-14 15:17:14,168107.0,162029.0,2023-05-31 23:48:28 EDT,...,La FMOQ dit aux médecins de famille de cesser ...,NaN,NaN,NaN,NaN,-64,2023,2023-05,1,2023-05-31
1,Le Journal de Montréal,jdemontreal,100064702091893,MEDIA_NEWS_COMPANY,CA,Des nouvelles accessibles et complètes: sports...,2011-06-09 20:52:08,739946.0,763966.0,2023-05-31 23:40:02 EDT,...,L'acteur américain Danny Masterson reconnu cou...,L’acteur américain Danny Masterson a été recon...,NaN,NaN,NaN,"1,17",2023,2023-05,82,2023-05-31
2,Le Journal de Québec,JdeQuebec,100064629642245,MEDIA_NEWS_COMPANY,CA,Des nouvelles accessibles et complètes: sports...,2010-10-05 19:04:37,453014.0,472544.0,2023-05-31 23:40:00 EDT,...,La NASA a tenu sa première réunion publique su...,Lors d’une réunion publique peu commune à Wash...,NaN,NaN,NaN,"-6,56",2023,2023-05,9,2023-05-31
3,CKVL FM,radiockvl,100063706583111,RADIO_STATION,CA,CKVL est la radio du grand Sud-Ouest de l'île ...,2013-01-15 20:40:07,8230.0,8940.0,2023-05-31 23:30:01 EDT,...,NaN,NaN,NaN,NaN,NaN,"-0,93",2023,2023-05,1,2023-05-31
4,Le Soleil,lesoleildequebec,100063641827245,MEDIA_NEWS_COMPANY,CA,Le Soleil est le média de référence en informa...,2011-01-20 21:20:29,108653.0,122736.0,2023-05-31 23:29:00 EDT,...,"La triple couronne dans la mire, les Remparts ...",NaN,NaN,NaN,NaN,"-2,94",2023,2023-05,16,2023-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780939,Le Soleil,lesoleildequebec,100063641827245,MEDIA_NEWS_COMPANY,CA,Le Soleil est le média de référence en informa...,2011-01-20 21:20:29,99216.0,114262.0,2022-07-01 00:33:43 EDT,...,Québec Mérite Mieux porte plainte pour vol d'i...,NaN,NaN,NaN,NaN,"5,3",2022,2022-07,350,2022-07-01
780940,La Voix de l'Est,lavoixdelest,100063772922658,TOPIC_NEWSPAPER,CA,La Voix de l'Est\nwww.lavoixdelest.ca | http:/...,2011-01-17 19:00:32,32443.0,35083.0,2022-07-01 00:29:09 EDT,...,"L’île des Serpents, une victoire ukrainienne s...",NaN,NaN,NaN,NaN,"-16,5",2022,2022-07,2,2022-07-01
780941,La Presse,LaPresseFB,100064750746253,NEWS_SITE,CA,Média d’information qui regroupe la plus grand...,2009-04-02 03:00:46,687996.0,714986.0,2022-07-01 00:09:04 EDT,...,Le procès de Brittney Griner en Russie suscite...,Américaine arrêtée pour des motifs légitimes e...,NaN,NaN,NaN,"-2,49",2022,2022-07,37,2022-07-01
780942,Le Journal de Montréal,jdemontreal,100064702091893,MEDIA_NEWS_COMPANY,CA,Des nouvelles accessibles et complètes: sports...,2011-06-09 20:52:08,721134.0,727979.0,2022-07-01 00:01:07 EDT,...,Moscou annonce un nouveau «rideau de fer»,L'OTAN a achevé jeudi son sommet à Madrid sur ...,NaN,NaN,NaN,"-4,76",2022,2022-07,37,2022-07-01


Combien de publications chaque média a-t-il publié au cours de cette période?

Voici le top-60.

In [25]:
fb["Page Name"].value_counts().head(60)

La Presse                             30123
Le Journal de Montréal                25280
Le Journal de Québec                  24892
Montreal Gazette                      21550
La Voix de l'Est                      19567
Le Nouvelliste                        19037
TVA Nouvelles                         18163
Le Soleil                             17109
MSN Québec                            16117
Global Montreal                       15758
Le Devoir                             15180
RDS                                   14511
Le Droit                              13791
Radio-Canada Information              13556
La Tribune                            13158
Le Quotidien                          12710
CityNews Montreal                     10598
Les Affaires                          10086
Narcity Québec                         9700
Journal Métro                          9517
www.lanouvelle.net                     9344
CJAD 800 Montreal                      7816
MTL Blog                        

Combien de médias ont été actifs dans Facebook à chacun des 17 mois couverts par cette période?

On voit, grâce à la méthode `.nunique()`, qu'il n'y en avait pas tous les mois 309.

In [24]:
fb.groupby(by="mois")["Page Name"].nunique()

mois
2022-01    292
2022-02    296
2022-03    297
2022-04    295
2022-05    294
2022-06    295
2022-07    288
2022-08    292
2022-09    297
2022-10    297
2022-11    297
2022-12    293
2023-01    292
2023-02    291
2023-03    294
2023-04    295
2023-05    278
Name: Page Name, dtype: int64

Je crée, ci-dessous, une nouvelle colonne pour identifier le jour au cours duquel chaque *post* a été publié.

In [17]:
fb["jour"] = fb["Post Created"].str[:10]

L'ensemble des jours possibles est ensuite placé dans une liste qui servira à l'étape suivante.

In [26]:
numpy.sort(fb.jour.unique())

array(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
       '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
       '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12',
       '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16',
       '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20',
       '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24',
       '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28',
       '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01',
       '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05',
       '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09',
       '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13',
       '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17',
       '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21',
       '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25',
       '2022-02-26', '2022-02-27', '2022-02-28', '2022-03-01',
       '2022-03-02', '2022-03-03', '2022-03-04', '2022-

On lit le fichier CSV réalisé dans le [carnet précédent](FB_300médias_parMois.ipynb) à l'aide du module CSV.

In [27]:
f = open("FB_22_23.csv")
posts = csv.reader(f)
next(posts)

['Page Name',
 'User Name',
 'Facebook Id',
 'Page Category',
 'Page Admin Top Country',
 'Page Description',
 'Page Created',
 'Likes at Posting',
 'Followers at Posting',
 'Post Created',
 'Post Created Date',
 'Post Created Time',
 'Type',
 'Total Interactions',
 'Likes',
 'Comments',
 'Shares',
 'Love',
 'Wow',
 'Haha',
 'Sad',
 'Angry',
 'Care',
 'Video Share Status',
 'Is Video Owner?',
 'Post Views',
 'Total Views',
 'Total Views For All Crossposts',
 'Video Length',
 'URL',
 'Message',
 'Link',
 'Final Link',
 'Image Text',
 'Link Text',
 'Description',
 'Sponsor Id',
 'Sponsor Name',
 'Sponsor Category',
 'Overperforming Score (weighted  —  Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )',
 'année',
 'mois',
 'inter',
 'jour']

Je crée ensuite une liste de mots, surtout des en anglais, que je souhaite exclure de l'analyse parce qu'ils ne sont pas significatifs. Ce n'est pas parce qu'ils sont en anglais qu'ils ne sont pas significatifs, mais parce qu'ils ne sont pas porteurs de sens dans le contexte actuel (pages Facebook de médias d'information).

In [34]:
motsNot = ["the", "in", "to", "and", "of", "|", "for", "s", "t", "full", "story", "tap", "click", "re-share",
               "link", "more", "at", "our", "follow", "us", "don'", "forget", "website", "like", "is", "post", "by"]

La cellule ci-dessous semble complexe, mais ce qu'on y fait est facile à décortiquer.

En somme, pour chaque jour dans la période étudiée, on lit le texte contenu dans tous les *posts* publiés ce jour-là. On met les textes de tous les *posts* dans la même variable (`texte`), qu'on nettoie à l'aide de quelques `.replace()`.

On traite ensuite ce texte avec la variable `tal` créée ci-dessus, ce qui permet d'extraire quatre ensembles:
* Les **mots seuls** (les lemmes seuls, en fait)
* Les paires de lemmes (les **bigrammes** de lemmes, en fait)
* Les **emojis**
* Les **mots-clics** (hashtags)

On dénombre les éléments de chacun de ces ensembles. Par exemple, les cinq bigrammes les plus rencontrés dans les publications Facebook du 24 juin 2022 ont été:
* fête national, 243 occurrences
* cour suprême, 93 occurrences
* national québec, 74 occurrences
* 24 juin, 63 occurrences
* droit avortement, 61 occurrences

Cela tombe sous le sens puisque ce jour-là, Fête nationale du Québec, on a appris que [la Cour suprême des États-Unis restreignait le droit à l'avortement](https://www.lapresse.ca/international/etats-unis/2022-06-24/droit-a-l-avortement/la-cour-supreme-invalide-l-arret-roe-v-wade.php).

La liste des éléments et leur nombre d'occurrences est enfin consigné dans quatre fichiers CSV distincts.

In [43]:
for jour in numpy.sort(fb.jour.unique()):
    
    f = open("FB_22_23.csv")
    posts = csv.reader(f)
    next(posts)
    
    texte = ""
    for post in posts:
        if post[-1] == jour:
            texte = texte + post[30] + " " + post[33] + " " + post[34] + " " + post[35] + " "
#             print(post[30])
#             print(post[33])
#             print(post[34])
#             print(post[35])
#             print("*****")
    
#     print(jour, len(texte))
            
    texte = texte.replace("\n", " ").replace("’", "'").replace("\xa0", " ").replace("\u2060", " ").replace("\u2009", " ").replace("\u2063", " ")
    
    while "  " in texte:
        texte = texte.replace("  "," ").strip()
    print(jour,len(texte))
    
    doc = tal(texte)
# #     print(len(doc))
    
    lemmes = [token.lemma_.lower() for token in doc if token.is_stop == False and token.is_punct == False and token.lemma_.lower() not in motsNot and len(token.text) > 1 and "/" not in token.text and "@" not in token.text]
#     print(mois,len(lemmes))
    
    freq = Counter(lemmes)
    print(freq.most_common(10))
    for n in freq.most_common():
#         print([jour[:7],n[0],n[1]])
        daft = open("FB_motsSeuls.csv", "a")
        punk = csv.writer(daft)
        punk.writerow([jour[:7],n[0],n[1]])
    
    bigrammes = []
    
    for index, lemme in enumerate(lemmes[:-1]):
        bigrammes.append("{} {}".format(lemmes[index], lemmes[index+1]))
    
    freq = Counter(bigrammes)
    print(freq.most_common(10))
    for n in freq.most_common():
#         print([jour[:7],n[0],n[1]])
        daft = open("FB_bigrammes.csv", "a")
        punk = csv.writer(daft)
        punk.writerow([jour[:7],n[0],n[1]])
#         break
    
    emojis = []

    for token in doc:
        if token.is_stop == False and token.is_punct == False and len(token.text) == 1:
            emojis.append(token.text)
    
    freq = Counter(emojis)
    print(freq.most_common(10))
    for n in freq.most_common():
#         print([jour[:7],n[0],n[1]])
        daft = open("FB_emojis.csv", "a")
        punk = csv.writer(daft)
        punk.writerow([jour[:7],n[0],n[1]])

2022-01-01 238374
[('année', 378), ('2021', 228), ('2022', 220), ('nouveau', 168), ('québec', 146), ('an', 144), ('covid-19', 137), ('bon', 134), ('être', 105), ('santé', 87)]
[('année 2022', 80), ('bon année', 71), ('artiste québécois', 36), ('bye bye', 35), ('nouveau année', 34), ('nouveau cas', 32), ('jour an', 30), ('monde 2021', 30), ('retenir attention', 28), ('17 122', 26)]
[('|', 182), ('s', 60), ('$', 39), ('5', 37), ('🥳', 31), ('2', 27), ('️', 26), ('h', 22), ('1', 18), ('✨', 18)]
2022-01-02 245676
[('québec', 188), ('covid-19', 165), ('année', 164), ('nouveau', 141), ('2021', 125), ('cas', 123), ('2022', 117), ('être', 109), ('an', 100), ('janvier', 96)]
[('nouveau cas', 53), ('cas covid-19', 39), ('15 845', 30), ('janvier 2022', 29), ('845 nouveau', 29), ('année 2022', 23), ('chu québec', 22), ('antonio brown', 20), ('prince andrew', 19), ('année 2021', 19)]
[('|', 229), ('2', 63), ('s', 56), ('$', 42), ('5', 29), ('3', 27), ('8', 23), ('6', 20), ('4', 18), ('1', 16)]
2022-

### Bigrammes

Le fichier de bigrammes est tellement important (15 millions de lignes) qu'il nécessite un traitement particulier.

In [47]:
big = pan.read_csv("FB_bigrammes.csv", names = ["mois", "bigramme", "nb"], low_memory = False)

In [48]:
big

,mois,bigramme,nb
0,2022-01,année 2022,80
1,2022-01,bon année,71
2,2022-01,artiste québécois,36
3,2022-01,bye bye,35
4,2022-01,nouveau année,34
...,...,...,...
15945342,2023-05,venir projet,1
15945343,2023-05,projet terminal,1
15945344,2023-05,terminal lng,1
15945345,2023-05,lng canada,1


Le tableau croisé ci-dessous (avec la méthode `.pivot_table`) permet d'identifier les bigrammes les plus couramment rencontrés dans les publications Facebook des médias du Québec.

In [60]:
# table = pan.pivot_table(big, values="nb", columns="mois", index="bigramme")
table = pan.pivot_table(big, values="nb", index="bigramme", aggfunc=numpy.sum)

In [61]:
table.sort_values(by="nb", ascending=False).head(60)

,nb
bigramme,
françois legault,9401
nouvelle union,9314
avenir érable,9304
union avenir,9295
fin semaine,9074
sûreté québec,8767
photo from,7793
national globalnews.ca,7466
centre ville,7145


In [51]:
big.mois.value_counts()

2022-06    1034379
2022-03    1025872
2022-05    1025828
2022-11     984534
2023-03     980846
2022-04     980076
2022-10     965066
2022-09     962880
2023-05     934447
2022-07     912209
2022-08     903886
2022-02     896136
2023-02     887779
2023-04     881330
2022-01     862130
2022-12     859304
2023-01     848645
Name: mois, dtype: int64

Enfin, l'opération ci-dessous permet de produire un tableau de l'activité Facebook des 294 pages médiatiques qui ont publié des contenus en 2022-23.

In [84]:
table2 = pan.pivot_table(fb,
                         values=[
                             "inter",
                             "URL",
                             "Likes at Posting",
                             "Followers at Posting",
                             "Post Views",
                             "Total Views",
                             "Likes",
                             "Comments",
                             "Shares",
                             "Love",
                             "Wow",
                             "Haha",
                             "Sad",
                             "Angry",
                             "Care"
                         ],
                         aggfunc={
                             "URL": "count",
                             "inter": numpy.sum,
                             "Likes at Posting": numpy.max,
                             "Followers at Posting": numpy.max,
                             "Post Views": numpy.sum,
                             "Total Views": numpy.sum,
                             "Likes": numpy.sum,
                             "Comments": numpy.sum,
                             "Shares": numpy.sum,
                             "Love": numpy.sum,
                             "Wow": numpy.sum,
                             "Haha": numpy.sum,
                             "Sad": numpy.sum,
                             "Angry": numpy.sum,
                             "Care": numpy.sum,
                             
                         },
#                          index=["Page Name", "User Name", "Facebook Id", "Page Created"]
                         index=["Page Name", "Facebook Id", "Page Created"]
                        )
table2

,,,Angry,Care,Comments,Followers at Posting,Haha,Likes,Likes at Posting,Love,Post Views,Sad,Shares,Total Views,URL,Wow,inter
Page Name,Facebook Id,Page Created,,,,,,,,,,,,,,,
"104,7 Outaouais",100063772441457,2012-08-15 16:36:10,15385,8537,105283,52596.0,27799,91145,46759.0,16235,329800,29410,104623,76499095,3969,12531,410948
106.9 FM Mauricie,100063670955241,2012-08-15 16:10:02,3695,2389,28489,22416.0,4190,27596,20866.0,5928,185049,3868,29364,9691042,1298,4001,109520
107.7 Estrie,100063589332484,2012-08-15 16:24:20,2539,932,45731,27683.0,4550,20711,27127.0,3374,88076,2783,8665,124901,2710,2015,91300
24 heures,509839125694576,2012-10-16 18:28:38,7535,7424,71348,76376.0,30844,116322,35144.0,18528,1529659,8535,38801,1778472,3458,8859,308196
"98,5 FM",100053947105104,2010-07-09 14:06:39,15624,13642,254546,119016.0,45044,129381,99322.0,27612,412187,20407,28554,484535,4541,7755,542565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
www.zone911.com,100064662494741,2009-06-15 22:00:21,15235,6135,86696,142286.0,17225,82364,137326.0,9930,57657,86698,155948,72918,1932,49275,509506
Écho de Cantley,100064563524801,2010-05-03 21:54:32,10,60,122,1142.0,0,1252,1047.0,112,1328,27,414,2973939,1482,45,2042
Écho de La Tuque,100063500232987,2010-01-12 20:58:02,472,1509,9226,10894.0,411,31529,10276.0,8627,50239,1718,16867,104687,1373,1798,72157


In [85]:
table2.to_csv("FB_médias_2022-2023.csv")

Une dernière opération pour identifier le nombre minimum de *followers* et de personnes qui aiment chaque page

In [86]:
table3 = pan.pivot_table(fb,
                         values=[
                             "Likes at Posting",
                             "Followers at Posting"
                         ],
                         aggfunc={
                             "Likes at Posting": numpy.min,
                             "Followers at Posting": numpy.min
                             
                         },
#                          index=["Page Name", "User Name", "Facebook Id", "Page Created"]
                         index=["Page Name", "Facebook Id", "Page Created"]
                        )
table3

,,,Followers at Posting,Likes at Posting
Page Name,Facebook Id,Page Created,,
"104,7 Outaouais",100063772441457,2012-08-15 16:36:10,49047.0,44700.0
106.9 FM Mauricie,100063670955241,2012-08-15 16:10:02,20638.0,20000.0
107.7 Estrie,100063589332484,2012-08-15 16:24:20,26785.0,26210.0
24 heures,509839125694576,2012-10-16 18:28:38,49243.0,29091.0
"98,5 FM",100053947105104,2010-07-09 14:06:39,108734.0,91971.0
...,...,...,...,...
www.zone911.com,100064662494741,2009-06-15 22:00:21,133462.0,130216.0
Écho de Cantley,100064563524801,2010-05-03 21:54:32,1043.0,942.0
Écho de La Tuque,100063500232987,2010-01-12 20:58:02,9791.0,9128.0


In [87]:
table3.to_csv("FB_médias_2022-2023-2.csv")

Une dernière, dernière, opération pour isoler le nombre de vidéos diffusées par chacun des médias de mon échantillon.

In [80]:
fb.groupby(by="Page Name").Type.value_counts()

Page Name                           Type               
104,7 Outaouais                     Link                   2730
                                    Photo                  1136
                                    Native Video             61
                                    Status                   19
                                    Live Video Complete      17
                                                           ... 
Écho de La Tuque                    YouTube                   1
Échos Montréal                      Photo                    17
Éditions du Journal de l'assurance  Link                   1093
                                    Photo                    40
                                    Status                    2
Name: Type, Length: 1489, dtype: int64

In [81]:
fb.groupby(by="Page Name").Type.value_counts().to_csv("FB_videos.csv")

In [82]:
fb.groupby(by="Page Name").URL.value_counts().to_csv("FB_urls.csv")

In [90]:
fb.groupby(by="User Name").URL.value_counts().to_csv("FB_urls2.csv")

In [93]:
fb.groupby(by=["User Name","Page Name"]).Type.value_counts()

User Name         Page Name        Type        
1019fm.net        CHAI 101.9 FM    Photo             39
                                   Status            39
                                   Link              13
                                   Native Video       2
1047fm.Outaouais  104,7 Outaouais  Link            2730
                                                   ... 
zone911           www.zone911.com  Native Video       4
zoneeconomie      Zone économie    Link             357
                                   Native Video     341
                                   Photo            277
                                   Status            22
Name: Type, Length: 1448, dtype: int64

In [94]:
fb.groupby(by=["User Name","Page Name"]).Type.value_counts().to_csv("FB_videos2.csv")